# Semantic Segmentation Input

## Install Dependencies

In [7]:
!pip install -r requirements.txt

## Setup Imports

In [8]:
from auxiliary.nifti.io import read_nifti
from rich import print as pprint
from panoptica import InputType, Panoptica_Evaluator, ConnectedComponentsInstanceApproximator, NaiveThresholdMatching

## Load Data

To demonstrate we use a reference and predicition of spine a segmentation without instances.

![semantic_figure](figures/semantic.png)

In [9]:
ref_masks = read_nifti("./spine_seg/semantic/ref.nii.gz")
pred_masks = read_nifti("./spine_seg/semantic/pred.nii.gz")

To use your own data please replace the example data with your own data.

In ordner to successfully load your data please use NIFTI files  and the following file designation within the "semantic" folder: 

```panoptica/spine_seg/semantic/```

- Reference data ("ref.nii.gz")
- Prediction data ("pred.nii.gz")


## Run Evaluation

In [10]:
evaluator = Panoptica_Evaluator(
    expected_input=InputType.SEMANTIC,
    instance_approximator=ConnectedComponentsInstanceApproximator(),
    instance_matcher=NaiveThresholdMatching(),
)

## Inspect Results
The results object allows access to individual metrics and provides helper methods for further processing


In [11]:
# print all results
result, _ = evaluator.evaluate(pred_masks, ref_masks, verbose=False)["ungrouped"]
print(result)

────────────────────────────────────────── Thank you for using panoptica ──────────────────────────────────────────

Please support our development by citing

https://github.com/BrainLesion/panoptica#citation -- Thank you!

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────


+++ MATCHING +++
Number of instances in reference (num_ref_instances): 87
Number of instances in prediction (num_pred_instances): 89
True Positives (tp): 73
False Positives (fp): 16
False Negatives (fn): 14
Recognition Quality / F1-Score (rq): 0.8295454545454546

+++ GLOBAL +++
Global Binary Dice (global_bin_dsc): 0.9731641527805414
Global Binary Centerline Dice (global_bin_cldsc): 0.9623567366028574
Global Binary Average Symmetric Surface Distance (global_bin_assd): 0.15559863304671395
Global Binary Relative Volume Difference (global_bin_rvd): 0.0021710740333888085

+++ INSTANCE +++
Segmentation Quality IoU (sq): 0.7940127477906024 +- 0.11547745015679488
Panoptic Quality IoU (pq): 0.6586696657808406
Segmentation Quality Dsc (sq_dsc): 0.8802182546605446 +- 0.07728416427007168
Panoptic Quality Dsc (pq_dsc): 0.7301810521615881
Segmentation Quality Assd (sq_assd): 0.2057371092494465 +- 0.1398348236766068
Segmentation Quality Relative Volume Difference (sq_rvd): 0.011340219860617232 +- 0.

In [12]:
# get specific metric, e.g. pq
pprint(f"{result.pq=}")

result.pq=0.6586696657808406

In [13]:
# get dict for further processing, e.g. for pandas
pprint("results dict: ", result.to_dict())

results dict: 
{
    'num_ref_instances': 87,
    'num_pred_instances': 89,
    'tp': 73,
    'fp': 16,
    'fn': 14,
    'prec': 0.8202247191011236,
    'rec': 0.8390804597701149,
    'rq': 0.8295454545454546,
    'global_bin_dsc': 0.9731641527805414,
    'global_bin_cldsc': 0.9623567366028574,
    'global_bin_assd': 0.15559863304671395,
    'global_bin_rvd': 0.0021710740333888085,
    'sq': 0.7940127477906024,
    'sq_std': 0.11547745015679488,
    'pq': 0.6586696657808406,
    'sq_dsc': 0.8802182546605446,
    'sq_dsc_std': 0.07728416427007168,
    'pq_dsc': 0.7301810521615881,
    'sq_assd': 0.2057371092494465,
    'sq_assd_std': 0.1398348236766068,
    'sq_rvd': 0.011340219860617232,
    'sq_rvd_std': 0.1217805112447998
}